# Imports,  Settings and Paths

In [1]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import time

In [2]:
from Code.Models.c7o2h10_model import DeepPotential, train, normalize, backtransform

In [3]:
data_path = './Dataset/c7o2h10_X.npy'
label_path = './Dataset/c7o2h10_Y.npy'
model_path = './ModelCheckpoints/c7o2h10/ '

# Read the Data

In [4]:
X_train_npy = np.load(data_path)
Y_train_npy = np.load(label_path) * -1
X_train_npy.shape

(404000, 19, 72)

In [5]:
Y_train_npy[:10]

array([11504.47279925, 11503.90459319, 11503.87676487, 11503.74995889,
       11503.70252929, 11504.1727455 , 11503.6886774 , 11504.1181435 ,
       11504.30685716, 11504.02239435])

### Check for NaN values

In [6]:
use_ids = np.where(np.isnan(X_train_npy).sum(axis=2).sum(axis=1)==0)
print('{} Datapoints with NaN values found.'.format(X_train_npy.shape[0] - len(use_ids[0])))

0 Datapoints with NaN values found.


In [7]:
X_train_npy = X_train_npy[use_ids]
Y_train_npy = Y_train_npy[use_ids]
X_train_npy.shape

(404000, 19, 72)

### Shuffle the Dataset

In [8]:
shuffle = np.arange(X_train_npy.shape[0])
np.random.shuffle(shuffle)
X_train_npy = X_train_npy[shuffle]
Y_train_npy = Y_train_npy[shuffle]

### Normalize Dataset

In [9]:
#Y_min = Y_train_npy.min()
#Y_train_npy -= Y_min
#Y_max = Y_train_npy.max()
#Y_train_npy /= Y_max

In [10]:
#print('Mean: {}\nVariance: {}'.format(Y_train_npy.mean(), Y_train_npy.var()))

### Create Pytorch Variables

In [11]:
X_data = Variable(torch.Tensor(X_train_npy).cuda())
Y_data = Variable(torch.Tensor(Y_train_npy).cuda(), requires_grad=False)

# Train Model

## Do the learning

In [ ]:
init_deep_pot = DeepPotential().cuda()
init_optim = torch.optim.Adam(init_deep_pot.parameters())
deep_pot, optim = train(init_deep_pot, init_optim, X_data, Y_data,
                        300, 128, (0.01, 0.96, 1.5),
                        checkpoint_path=model_path, print_every=100)


Epoch: 0	learning rate: 0.01
---
total: 0.0 %	current epoch: 4.0 %	loss: 0.190811	time estimate: 884.3 min
total: 0.0 %	current epoch: 7.9 %	loss: 0.191812	time estimate: 808.0 min
total: 0.0 %	current epoch: 11.9 %	loss: 0.190847	time estimate: 648.1 min
total: 0.1 %	current epoch: 15.8 %	loss: 0.19018	time estimate: 568.2 min
total: 0.1 %	current epoch: 19.8 %	loss: 0.191548	time estimate: 520.2 min
total: 0.1 %	current epoch: 23.8 %	loss: 0.190232	time estimate: 488.3 min
total: 0.1 %	current epoch: 27.7 %	loss: 0.190584	time estimate: 465.4 min
total: 0.1 %	current epoch: 31.7 %	loss: 0.191324	time estimate: 447.9 min
total: 0.1 %	current epoch: 35.7 %	loss: 0.19094	time estimate: 434.4 min
total: 0.1 %	current epoch: 39.6 %	loss: 0.191271	time estimate: 423.5 min
total: 0.1 %	current epoch: 43.6 %	loss: 0.192279	time estimate: 414.6 min
total: 0.2 %	current epoch: 47.5 %	loss: 0.190651	time estimate: 407.1 min
total: 0.2 %	current epoch: 51.5 %	loss: 0.191933	time estimate: 402.8

In [ ]:
test_size = 10000

In [ ]:
deep_pot = DeepPotential().cuda()
deep_pot.load_state_dict(torch.load('./ModelCheckpoints/epoch_13'))
Y_result = deep_pot.forward(X_data[-test_size:])

In [ ]:
result = backtransform(np.array(Y_result.data.tolist()).reshape(test_size), Y_data.min().item(), Y_data.max().item())

In [ ]:
test = np.array(Y_data[-test_size:].data.tolist()).reshape(test_size)

In [ ]:
np.mean(np.abs(test - result))

## Test the model
### Mean Absolute Error
The desired accuracy is about 0.04 eV

In [ ]:
mae = np.abs(deep_pot.forward(X_data[-500:]).data.numpy().reshape(500) - Y_data[-500:].data.numpy().reshape(500)).mean()
print('The nural network reaches a mean absolute error of {} eV'.format(mae))

### Small test sample

In [ ]:
deep_pot.forward(X_data[-10:])

In [ ]:
Y_data[-10:]

## Save the Model Parameters

In [ ]:
torch.save(deep_pot.state_dict(), model_path)

## Continue learning

In [ ]:
deep_pot = DeepPotential()
deep_pot.load_state_dict(torch.load(model_path))
optim = torch.optim.Adam(deep_pot.parameters(), lr=1e-2)

In [ ]:
deep_pot, optim = train(deep_pot, optim, X_data, Y_data, 1000, 512, (0.001, 0.96, 1.5), use_for_train=0.9, print_every=10)